In [1]:
import os
import time
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
# 1、加载数据，包括训练集和测试集
cur_path = os.getcwd() + os.sep
train_path = cur_path + "train.csv"
test_path = cur_path + "test.csv"
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [3]:
# 2、简单看看数据，展示不全，可以上excel看
train_data.head(5)

,user_id,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,1374,58,No,Travel_Rarely,605,Sales,21,3,Life Sciences,1,...,3,80,1,29,2,2,1,0,0,0
1,1092,45,No,Travel_Rarely,950,Research & Development,28,3,Technical Degree,1,...,4,80,1,8,3,3,5,4,0,3
2,768,40,No,Travel_Rarely,300,Sales,26,3,Marketing,1,...,2,80,1,8,3,2,7,7,7,5
3,569,36,No,Non-Travel,1434,Sales,8,4,Life Sciences,1,...,2,80,0,10,1,3,10,7,0,9
4,911,25,Yes,Travel_Frequently,599,Sales,24,1,Life Sciences,1,...,4,80,0,1,4,3,1,0,1,0


In [4]:
# 3、简单分析数据缺失情况
def get_na_rate(data_df): 
    column_list = data_df.columns
    na_rate_dict = dict()
    for column in column_list:
        na_rate_dict.update({column:data_df[column].isna().sum()})
    return na_rate_dict

print(get_na_rate(train_data))

{'WorkLifeBalance': 0, 'MaritalStatus': 0, 'user_id': 0, 'TrainingTimesLastYear': 0, 'Gender': 0, 'YearsInCurrentRole': 0, 'EducationField': 0, 'YearsWithCurrManager': 0, 'EmployeeNumber': 0, 'JobRole': 0, 'NumCompaniesWorked': 0, 'MonthlyRate': 0, 'RelationshipSatisfaction': 0, 'HourlyRate': 0, 'Department': 0, 'PercentSalaryHike': 0, 'OverTime': 0, 'Attrition': 0, 'JobSatisfaction': 0, 'Age': 0, 'DistanceFromHome': 0, 'YearsAtCompany': 0, 'JobInvolvement': 0, 'Education': 0, 'DailyRate': 0, 'JobLevel': 0, 'EmployeeCount': 0, 'BusinessTravel': 0, 'PerformanceRating': 0, 'TotalWorkingYears': 0, 'YearsSinceLastPromotion': 0, 'EnvironmentSatisfaction': 0, 'StockOptionLevel': 0, 'StandardHours': 0, 'Over18': 0, 'MonthlyIncome': 0}


In [5]:
# 4、看看哪些数据要做数值化处理
# print(train_data.dtypes) # 看完以后简单归归类
def get_obj_rate(data_df):
    obj_rate_dict = dict()
    useless_columns = list()
    data_length = data_df.shape[0]
    for column in data_df.columns:
        unique_count = len(data_df[column].unique())
        if unique_count == 1 or unique_count == data_length:
            useless_columns.append(column)   # 找到明显无效的特征列
            continue
        if isinstance(data_df[column][0],str):
            obj_rate_dict.update({column: (unique_count, unique_count/(1.0*data_length))})
    return obj_rate_dict, useless_columns
obj_rate_dict, useless_columns = get_obj_rate(train_data)
print(obj_rate_dict)
print(useless_columns)

{'Department': (3, 0.002551020408163265), 'OverTime': (2, 0.0017006802721088435), 'BusinessTravel': (3, 0.002551020408163265), 'JobRole': (9, 0.007653061224489796), 'MaritalStatus': (3, 0.002551020408163265), 'Gender': (2, 0.0017006802721088435), 'Attrition': (2, 0.0017006802721088435), 'EducationField': (6, 0.00510204081632653)}
['user_id', 'EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours']


In [6]:
# 5、对那些完全无效信息，先去除掉
def remove_useless_data(data_df_train, data_df_test, useless_columns):
    for column in useless_columns:
        data_df_train.drop(column, axis=1, inplace=True)
        data_df_test.drop(column, axis=1, inplace=True)
        
remove_useless_data(train_data, test_data, useless_columns)

In [7]:
# 6、数值化一下，因为我们要使用的都是树模型，这里我们都用label encoding，具体原因可以看LightGBM这篇订阅号
def to_numeric(data_df_train, data_df_test, columns_dict, label_column):    
    for column in columns_dict.keys():
        if column == label_column:
            data_df_train[column]=data_df_train[column].map(lambda x:1 if x=='Yes' else 0)
            continue
        lbe = LabelEncoder()
        data_df_train[column] = lbe.fit_transform(data_df_train[column])
        data_df_test[column] = lbe.transform(data_df_test[column])
to_numeric(train_data, test_data, obj_rate_dict, label_column="Attrition")
train_data.head(5)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,58,0,2,605,2,21,3,1,4,0,...,3,3,1,29,2,2,1,0,0,0
1,45,0,2,950,1,28,3,5,4,1,...,4,4,1,8,3,3,5,4,0,3
2,40,0,2,300,2,26,3,2,3,1,...,3,2,1,8,3,2,7,7,7,5
3,36,0,0,1434,2,8,4,1,1,1,...,3,2,0,10,1,3,10,7,0,9
4,25,1,1,599,2,24,1,1,3,1,...,3,4,0,1,4,3,1,0,1,0


In [8]:
# 7、数据不做深度剖析，这里只做到这里，然后分一下训练集和验证集
def get_train_valid_data(features, labels, test_size=0.2, random_state=666):
    x_train, x_valid, y_train, y_valid = train_test_split(features, labels, test_size=test_size, random_state=random_state)
    return x_train, x_valid, y_train, y_valid

label_column = "Attrition"
x_train, x_valid, y_train, y_valid = get_train_valid_data(train_data.drop(label_column,axis=1), train_data[label_column])

In [9]:
# 分别对训练集，验证集，以及测试集进行一式五份的拷贝，因为我们下面要用五个模型分别跑一下
x_train_gbdt, x_train_xgb, x_train_lgb, x_train_catb, x_train_ngb = x_train.copy(),x_train.copy(),x_train.copy(),x_train.copy(),x_train.copy()
x_valid_gbdt, x_valid_xgb, x_valid_lgb, x_valid_catb, x_valid_ngb = x_valid.copy(),x_valid.copy(),x_valid.copy(),x_valid.copy(),x_valid.copy()
y_train_gbdt, y_train_xgb, y_train_lgb, y_train_catb, y_train_ngb = y_train.copy(),y_train.copy(),y_train.copy(),y_train.copy(),y_train.copy()
y_valid_gbdt, y_valid_xgb, y_valid_lgb, y_valid_catb, y_valid_ngb = y_valid.copy(),y_valid.copy(),y_valid.copy(),y_valid.copy(),y_valid.copy()

test_gbdt, test_xgb, test_lgb, test_catb, test_ngb = test_data.copy(), test_data.copy(), test_data.copy(), test_data.copy(), test_data.copy()

In [10]:
# 准备好打分函数，这里我们只是统计准确率
from sklearn.metrics import accuracy_score
def get_scores(y_true, y_predict):
    accuracy = '%.4f' % accuracy_score(y_true, y_predict)
    return accuracy

In [11]:
# GBDT
from sklearn.ensemble import GradientBoostingRegressor   # 这里为了能获得更好的AUC，使用回归模型
start_time = time.time()
model = GradientBoostingRegressor(random_state=10)
model.fit(x_train_gbdt, y_train_gbdt)
y_valid_predict_gbdt = model.predict(x_valid_gbdt)
y_valid_predict_gbdt = pd.Series(y_valid_predict_gbdt)
y_valid_predict_gbdt = y_valid_predict_gbdt.map(lambda x:1 if x>=0.5 else 0)

# 验证集看一下得分效果
print("auc: {}".format(get_scores(y_valid_gbdt, y_valid_predict_gbdt)))

predict = model.predict(test_gbdt)

test_gbdt[label_column] = predict
# 转化为二分类输出
test_gbdt[label_column]=test_gbdt[label_column].map(lambda x:1 if x>=0.5 else 0)
test_gbdt[label_column].to_csv('submit_gbdt.csv')
print("total costs {} seconds".format(time.time()-start_time))

auc: 0.8644
total costs 0.2689990997314453 seconds


D:\softWare\Anaconda\lib\site-packages\ipykernel\__main__.py:18: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [12]:
# XGBoost
import xgboost as xgb
start_time = time.time()
x_train_xgb = xgb.DMatrix(x_train_xgb, label=y_train_xgb)
x_valid_xgb = xgb.DMatrix(x_valid_xgb, label=y_valid_xgb)
test_xgb_transformed = xgb.DMatrix(test_xgb)

param = {'boosting_type': 'gbdt',
         'objective': 'binary:logistic',  
         'eval_metric': 'auc',
         'eta': 0.01,
         'max_depth': 15,
         'colsample_bytree': 0.8,
         'subsample': 0.9,
         'subsample_freq': 8,
         'alpha': 0.6,
         'lambda': 0,
         }
model = xgb.train(param, x_train_xgb, evals=[(x_train_xgb, 'train'), (x_valid_xgb, 'valid')], num_boost_round=10000,
                  early_stopping_rounds=200, verbose_eval=100)
y_valid_predict_xgb = model.predict(x_valid_xgb)
y_valid_predict_xgb = pd.Series(y_valid_predict_xgb)
y_valid_predict_xgb = y_valid_predict_xgb.map(lambda x:1 if x>=0.5 else 0)

# 验证集看一下得分效果
print("auc: {}".format(get_scores(y_valid_xgb, y_valid_predict_xgb)))

predict = model.predict(test_xgb_transformed)
test_xgb[label_column] = predict
test_xgb[label_column]=test_xgb[label_column].map(lambda x:1 if x>=0.5 else 0)
test_xgb[label_column].to_csv('submit_xgb.csv')
print("total costs {} seconds".format(time.time()-start_time))

[00:34:32] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { boosting_type, subsample_freq } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.89608	valid-auc:0.72375
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 200 rounds.


D:\softWare\Anaconda\lib\site-packages\xgboost\__init__.py:29: FutureWarning: Python 3.5 support is deprecated; XGBoost will require Python 3.6+ in the near future. Consider upgrading to Python 3.6+.
  FutureWarning)


[100]	train-auc:0.99207	valid-auc:0.80742
[200]	train-auc:0.99876	valid-auc:0.81951
[300]	train-auc:0.99990	valid-auc:0.82642
[400]	train-auc:1.00000	valid-auc:0.83227
[500]	train-auc:1.00000	valid-auc:0.83347
[600]	train-auc:1.00000	valid-auc:0.83347
Stopping. Best iteration:
[466]	train-auc:1.00000	valid-auc:0.83440

auc: 0.8517
total costs 2.9670000076293945 seconds


D:\softWare\Anaconda\lib\site-packages\ipykernel\__main__.py:31: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [13]:
# lightGBM
import lightgbm as lgb
start_time = time.time()
x_train_lgb = lgb.Dataset(x_train_lgb, label=y_train_lgb)
x_valid_lgb_transformed = lgb.Dataset(x_valid_lgb, label=y_valid_lgb)

param = {'boosting_type': 'gbdt',
         'objective': 'binary',  
         'metric': 'auc',
         'learning_rate': 0.01,
         'max_depth': 15,
         'feature_fraction': 0.8,
         'bagging_fraction': 0.9,
         'bagging_freq': 8,
         'lambda_l1': 0.6,
         'lambda_l2': 0,
         # 'scale_pos_weight':k,
         # 'is_unbalance':True
         }
category = list(obj_rate_dict.keys())
category.remove(label_column)

model = lgb.train(param, x_train_lgb, valid_sets=[x_train_lgb, x_valid_lgb_transformed], num_boost_round=10000,
                  early_stopping_rounds=200, verbose_eval=200, categorical_feature=category)

y_valid_predict_lgb = model.predict(x_valid_lgb)
y_valid_predict_lgb = pd.Series(y_valid_predict_lgb)
y_valid_predict_lgb = y_valid_predict_lgb.map(lambda x:1 if x>=0.5 else 0)

# 验证集看一下得分效果
print("auc: {}".format(get_scores(y_valid_lgb, y_valid_predict_lgb)))

predict = model.predict(test_lgb)
test_lgb[label_column] = predict
test_lgb[label_column]=test_lgb[label_column].map(lambda x:1 if x>=0.5 else 0)
test_lgb[label_column].to_csv('submit_lgb.csv')
print("total costs {} seconds".format(time.time()-start_time))

Training until validation scores don't improve for 200 rounds


D:\softWare\Anaconda\lib\site-packages\lightgbm\basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[200]	training's auc: 0.986203	valid_1's auc: 0.812068
Early stopping, best iteration is:
[15]	training's auc: 0.918869	valid_1's auc: 0.819312
auc: 0.8390
total costs 0.5789990425109863 seconds


D:\softWare\Anaconda\lib\site-packages\ipykernel\__main__.py:38: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [14]:
# catBoost
import catboost as cb
model = cb.CatBoostClassifier(iterations=1000, 
                              depth=7, 
                              learning_rate=0.01, 
                              loss_function='Logloss', 
                              eval_metric='AUC',
                              logging_level='Verbose', 
                              metric_period=200
                             )
# 得到分类特征的列号
categorical_features_indices = []
for i in range(len(x_train_catb.columns)):
    if x_train_catb.columns.values[i] in category:
        categorical_features_indices.append(i)
print(categorical_features_indices)

model.fit(x_train_catb, y_train_catb, eval_set=(x_valid_catb, y_valid_catb), cat_features=categorical_features_indices)

y_valid_predict_catb = model.predict(x_valid_catb)
y_valid_predict_catb = pd.Series(y_valid_predict_catb)
y_valid_predict_catb = y_valid_predict_catb.map(lambda x:1 if x>=0.5 else 0)

# 验证集看一下得分效果
print("auc: {}".format(get_scores(y_valid_catb, y_valid_predict_catb)))

predict = model.predict(test_catb)
test_catb[label_column] = predict
test_catb[label_column]=test_catb[label_column].map(lambda x:1 if x>=0.5 else 0)
test_catb[label_column].to_csv('submit_catb.csv')
print("total costs {} seconds".format(time.time()-start_time))

[1, 3, 6, 8, 12, 14, 18]
0:	test: 0.5572169	best: 0.5572169 (0)	total: 81.5ms	remaining: 1m 21s
200:	test: 0.8347953	best: 0.8347953 (200)	total: 3.38s	remaining: 13.4s
400:	test: 0.8426369	best: 0.8426369 (400)	total: 6.8s	remaining: 10.2s
600:	test: 0.8438331	best: 0.8438331 (600)	total: 10.4s	remaining: 6.88s
800:	test: 0.8414407	best: 0.8438331 (600)	total: 14.1s	remaining: 3.5s
999:	test: 0.8373206	best: 0.8438331 (600)	total: 17.7s	remaining: 0us

bestTest = 0.8438330675
bestIteration = 600

Shrink model to first 601 iterations.
auc: 0.8517
total costs 19.11639404296875 seconds


D:\softWare\Anaconda\lib\site-packages\ipykernel\__main__.py:30: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [23]:
# ngBoost
import ngboost as ng
model = ng.NGBClassifier(n_estimators=1000,
                         learning_rate=0.01,
                         verbose=True,
                         verbose_eval=200
                         )

model.fit(x_train_ngb, y_train_ngb)

y_valid_predict_ngb = model.predict_proba(x_valid_ngb)[:,1]
y_valid_predict_ngb = pd.Series(y_valid_predict_ngb)
y_valid_predict_ngb = y_valid_predict_ngb.map(lambda x:1 if x>=0.5 else 0)

# 验证集看一下得分效果
print("auc: {}".format(get_scores(y_valid_ngb, y_valid_predict_ngb)))

# predict = model.predict(test_ngb)
predict = model.predict_proba(test_ngb)[:,1]   # 这里可以直接获取概率值，我们取其为1对应的概率
test_ngb[label_column] = predict
test_ngb[label_column]=test_ngb[label_column].map(lambda x:1 if x>=0.5 else 0)
test_ngb[label_column].to_csv('submit_ngb.csv')
print("total costs {} seconds".format(time.time()-start_time))

[iter {itr}] loss={loss:.4f} val_loss={val_loss:.4f} scale={scale:.4f} norm={grad_norm:.4f}
[iter {itr}] loss={loss:.4f} val_loss={val_loss:.4f} scale={scale:.4f} norm={grad_norm:.4f}
[iter {itr}] loss={loss:.4f} val_loss={val_loss:.4f} scale={scale:.4f} norm={grad_norm:.4f}
[iter {itr}] loss={loss:.4f} val_loss={val_loss:.4f} scale={scale:.4f} norm={grad_norm:.4f}
[iter {itr}] loss={loss:.4f} val_loss={val_loss:.4f} scale={scale:.4f} norm={grad_norm:.4f}
auc: 0.8559
total costs 578.911324262619 seconds


D:\softWare\Anaconda\lib\site-packages\ipykernel\__main__.py:22: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [ ]:
# 调参是一个及其复杂的过程，尤其要每个模型都到最佳。
# 这里就不逐一去最佳化调参了。